# 概要
* 複数インデックスのマージにいよいよトライする
* treeは自動で要約つけてくれるの？？めっちゃ便利やん
* 訳としては完璧やん...
* num_outputは2048も対応している→ダイでエラー発生
* 親の要約が変なのが惜しい
* b_indexがおかしい
* #prompt_helperで重複を許すようにすれば自然な日本語になるはず#！！
* そもそもこういう問題があるからチュートリアルはkeyword_tableを使っていたのか
* indexを作っただけではembeddingしていないことに注意！！
* 親がkeyword_tableで子をtreeにするべきなのでは？
* 多分これ以外は組み合わせるけど、組み合わせ的にうまく動かない
* 日本語のkeywordを取るには一旦graph化が必要（公式ドキュメントには記載ない？）
* 結局vector-> kw -> graphの公式通りなら（一応）動いた
* https://gpt-index.readthedocs.io/en/latest/how_to/composability.html#querying-the-top-level-index

# 参考
* そもそもtree構造が前提なのでtreeのindexを確認する
* https://gpt-index.readthedocs.io/en/latest/how_to/composability.html
* https://note.com/npaka/n/n5d4ff4380055
* https://note.com/oshizo/n/n137aaa2c29d4
* 日本語としてparseされない...

# 結局
* 一応ダイとドラゴンボールで動いてる
* ただ精度はいまいち
* そもそも下部構造のtreeの精度がいまいち
* 一応動いちょる
* ふぅ〜って感じ。

In [1]:
import os
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

In [2]:
#Jupyterでも実行できるのは知らんかった...
import logging
import sys
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG, force=True)

In [3]:
from gpt_index import LLMPredictor, GPTSimpleVectorIndex, SimpleDirectoryReader

In [4]:
# OpenAIのAPIのラッパー
from langchain.llms import OpenAI

# 利用例

In [5]:
from llama_index import (
    GPTSimpleVectorIndex, 
    GPTSimpleKeywordTableIndex, 
    GPTListIndex, 
    SimpleDirectoryReader,
    GPTTreeIndex
)

from llama_index import (
    GPTKeywordTableIndex, 
    SimpleDirectoryReader, 
    LLMPredictor,
    PromptHelper
)

from gpt_index.indices.knowledge_graph.base import GPTKnowledgeGraphIndex
from langchain import OpenAI

# define prompt helper
# set maximum input size
max_input_size = 4096
# set number of output tokens
num_output = 1024+512
# set maximum chunk overlap
max_chunk_overlap = 0
# separatorが"/n/n"だとチャンクサイズの計算でエラーになるっぽい。"。"の場合は動く模様。
prompt_helper = PromptHelper(max_input_size, num_output, max_chunk_overlap, separator="。", chunk_size_limit=512)

# define LLM
llm_predictor = LLMPredictor(llm=OpenAI(temperature=0, model_name="text-davinci-003", max_tokens=num_output))

# build index
documents = SimpleDirectoryReader('story/dragonball').load_data()
d_index = GPTSimpleVectorIndex(documents, prompt_helper=prompt_helper, llm_predictor=llm_predictor)

documents = SimpleDirectoryReader('story/dai').load_data()
b_index = GPTSimpleVectorIndex(documents, prompt_helper=prompt_helper, llm_predictor=llm_predictor)

DEBUG:root:> [SimpleDirectoryReader] Total files added: 1
DEBUG:root:> Adding chunk: 孫悟空少年編
地球の人里離れた山奥に住む尻尾の生えた少年・孫悟空はある日、西の都からやって来た...
DEBUG:root:> Adding chunk: 

その後、悟空は旅の途中に知り合った武術の達人・亀仙人の下で、後に親友となるクリリンと共に8...
DEBUG:root:> Adding chunk: 悟空は仇を討つため、道中に出会ったヤジロベーや仙猫カリンの協力を得て命を賭して潜在する力を引き...
DEBUG:root:> Adding chunk: 

サイヤ人編
ピッコロ（マジュニア）との闘いから約5年後、息子の孫悟飯を儲けて平和な日々を過...
DEBUG:root:> Adding chunk: 仲間の協力もあり、何とか辛勝し撤退させるが、多くの仲間を失うとともに、ピッコロの戦死により彼と...
DEBUG:root:> Adding chunk: フリーザの持つ圧倒的な力の前にはベジータやピッコロ、悟空すら歯が立たず仲間たちが次々と命を落と...
DEBUG:root:> Adding chunk: トランクスと名乗るその青年は、自分は未来からやってきたブルマとベジータの息子であることを悟空に...
DEBUG:root:> Adding chunk: 悟空らは天界にある1日で1年の修行が行えるも過酷な環境に晒される「精神と時の部屋」で修行し、強...
DEBUG:root:> Adding chunk: 天下一武道会の最中、悟空たちは界王よりもさらに高位の存在である界王神から、恐ろしい力を持つ魔人...
DEBUG:root:> Adding chunk: ドラゴンボールの協力もあり、地球・ナメック星・あの世の人々のエネルギーによって作り上げられた超...
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings
DEBUG:openai:api_version=None da

DEBUG:urllib3.util.retry:Converted retries value: 2 -> Retry(total=2, connect=None, read=None, redirect=None, status=None)
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.openai.com:443
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/engines/text-embedding-ada-002/embeddings HTTP/1.1" 200 82889
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings processing_ms=319 request_id=d1eea0de0c059dd4731c582c8352ebbd response_code=200
INFO:root:> [build_index_from_documents] Total LLM token usage: 0 tokens
INFO:root:> [build_index_from_documents] Total embedding token usage: 4371 tokens
DEBUG:root:> [SimpleDirectoryReader] Total files added: 1
DEBUG:root:> Adding chunk: 序章（デルパ! イルイル! / ダイ爆発!!!）
魔王ハドラー率いる魔王軍と勇者率いる勇者パー...
DEBUG:root:> Adding chunk: そこに勇者の家庭教師を名乗る謎の人物アバンとその弟子の魔法使いポップが現れ、島を破邪呪文・マホ...
DEBUG:root:> Adding chunk: アバンは弟子たちを護るために自己犠牲呪文・メガンテを使い、その命を散らすもハドラーは生き残り、...
DEBUG:root:> Adding chu

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/engines/text-embedding-ada-002/embeddings HTTP/1.1" 200 82889
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings processing_ms=153 request_id=fb635f205b78a3708d21e29dae02986c response_code=200
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings
DEBUG:openai:api_version=None data='{"input": ["\\u5e8f\\u7ae0\\uff08\\u30c7\\u30eb\\u30d1! \\u30a4\\u30eb\\u30a4\\u30eb! / \\u30c0\\u30a4\\u7206\\u767a!!!\\uff09 \\u9b54\\u738b\\u30cf\\u30c9\\u30e9\\u30fc\\u7387\\u3044\\u308b\\u9b54\\u738b\\u8ecd\\u3068\\u52c7\\u8005\\u7387\\u3044\\u308b\\u52c7\\u8005\\u30d1\\u30fc\\u30c6\\u30a3\\u306e\\u6226\\u3044\\u306f\\u3001\\u52c7\\u8005\\u30d1\\u30fc\\u30c6\\u30a3\\u306e\\u52dd\\u5229\\u3067\\u7d42\\u308f\\u308a\\u3001\\u9b54\\u738b\\u306e\\u914d\\u4e0b\\u3067\\u3042\\u3063\\u305f\\u30e2\\u30f3\\u30

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/engines/text-embedding-ada-002/embeddings HTTP/1.1" 200 91164
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings processing_ms=293 request_id=8639727b310a5154b3bef6516f9d6362 response_code=200
INFO:root:> [build_index_from_documents] Total LLM token usage: 0 tokens
INFO:root:> [build_index_from_documents] Total embedding token usage: 4404 tokens


In [33]:
b_index.docstore.docs

{'18482556-4c5f-4711-a484-06ad841dc8f7': SimpleIndexDict(text='\n十数年前、魔王ハドラー率いる魔王軍と勇者率いる勇者パーティの戦いは、勇者パーティの勝利で終わり、魔王の配下であったモンスターも魔王の支配から解き放たれ、世界に平和が訪れた。それから十数年後、デルムリン島に住む少年ダイは、勇者になることを夢見ていた。ある日、魔王が復活したことを察するブラスに訪れた謎の人物アバンとその弟子の魔法使いポップが現れ、島を破邪呪文・マホカトールで覆い、モンスターたちを魔王の支配から救った。アバンはダイの事を聞き、その将来性を見込んで島を訪れたのである。ダイはアバンの指導を受けて、短期間のうちに秘められた力を開花させていく。15年ぶりに復活したハドラーがアバン抹殺のためデルムリン島に来襲するが、ダイは額に浮き出た謎の紋章の力でパワーアップし、ハドラーを退ける。ダイは魔王軍を倒すべく、ポップとゴメちゃんと共に旅立ち、ロモス王国とパプニカ王国を占拠した魔王軍六大軍団と戦い、勝利を収める。\nダイは人間からの視線に苦悩していたが、その前に竜騎将バランが現れ、ダイが神々が作った伝説の存在「竜の騎士」であることを告げた。バランはダイに魔王軍に加わるよう促したが、ダイは拒否した。その後、ポップがメガンテを発動して犠牲になり、ダイの記憶が蘇った。レオナは世界各国の王に協力を呼びかけ、魔王軍への決戦の準備を進める。ダイ一行も新たな力を蓄え、ハドラーは妖魔司教ザボエラの手で自らを超魔生物にさせる。ダイ一行は魔王軍の本拠地である鬼岩城を破壊したが、バーンパレスには魔宮の門によって閉ざされていた。バランはダイ一行と共闘し、ハドラーと対峙するが、黒の核晶が起動してしまい、バランはダイたちを守るために全ての力を使って爆発を押さえこみながら命を落とした。\nダイ一行はバーンとの直接対決において圧倒的な力の差の前に敗北し、ハドラーがバーンに反逆し、攻撃を仕掛けた隙に脱出する。バーンはバーンパレスを飛行させ、世界各地への爆撃を実行するが、フローラの協力を得て辛うじて生き延びたダイ一行は、レオナを中心に大破邪呪文・ミナカトールを発動し、バーンパレスの動きを封じ突入する。更にハドラーと親衛騎団も乱入し、激闘を繰り広げる。ダイはハドラーを倒すが、キルバー

In [10]:
d_index.docstore.docs

{'f45ce7f9-f61f-481d-988a-7db2a702842b': SimpleIndexDict(text=None, doc_id='f45ce7f9-f61f-481d-988a-7db2a702842b', embedding=None, extra_info=None, nodes_dict={1609494147561157073: Node(text='孫悟空少年編\n地球の人里離れた山奥に住む尻尾の生えた少年・孫悟空はある日、西の都からやって来た少女・ブルマと出会う。そこで、7つ集めると神龍（シェンロン）が現れ、どんな願いでも一つだけ叶えてくれるというドラゴンボールの存在を、さらに育ての親である孫悟飯の形見として大切に持っていた球がその1つ「四星球（スーシンチュウ）」であることを知り、ブルマと共に残りのドラゴンボールを探す旅に出る。人さらいのウーロンや盗賊のヤムチャらを巻き込んだボール探しの末、世界征服を企むピラフ一味にボールを奪われ神龍を呼び出されるが、ウーロンがとっさに言い放った下らない願いを叶えてもらうことで一味の野望を阻止する', doc_id='5af9424b-879f-4b59-aa4f-f4715b35ec2a', embedding=None, extra_info=None, index=0, child_indices=set(), ref_doc_id='3c95a769-45fb-4b92-8c5c-5273bad79d1d', node_info={'start': 0, 'end': 296}), 7080135353598358373: Node(text='\n\nその後、悟空は旅の途中に知り合った武術の達人・亀仙人の下で、後に親友となるクリリンと共に8か月間にわたる修行を積み、その成果を確かめるために世界一の武術の達人を決める天下一武道会に出場し、変装して出場していた亀仙人に敗れるも準優勝を果たす。悟空は再び修行の旅へと出発し、ドラゴンボールの悪用を企むレッドリボン軍との闘いや、孫悟飯との再会などを経てさらに強さを増していく。さらに3年後の天下一武道会では、亀仙流のライバルである鶴仙流の天津飯（てんしんはん）と闘うが、あと一歩のところで敗れ、前回と同じく準優勝に終わる。\n\nピッコロ大魔王編\n天下

In [9]:
b_index.docstore.docs['18482556-4c5f-4711-a484-06ad841dc8f7'].text="""
十数年前、魔王ハドラー率いる魔王軍と勇者率いる勇者パーティの戦いは、勇者パーティの勝利で終わり、魔王の配下であったモンスターも魔王の支配から解き放たれ、世界に平和が訪れた。それから十数年後、デルムリン島に住む少年ダイは、勇者になることを夢見ていた。ある日、魔王が復活したことを察するブラスに訪れた謎の人物アバンとその弟子の魔法使いポップが現れ、島を破邪呪文・マホカトールで覆い、モンスターたちを魔王の支配から救った。アバンはダイの事を聞き、その将来性を見込んで島を訪れたのである。ダイはアバンの指導を受けて、短期間のうちに秘められた力を開花させていく。15年ぶりに復活したハドラーがアバン抹殺のためデルムリン島に来襲するが、ダイは額に浮き出た謎の紋章の力でパワーアップし、ハドラーを退ける。ダイは魔王軍を倒すべく、ポップとゴメちゃんと共に旅立ち、ロモス王国とパプニカ王国を占拠した魔王軍六大軍団と戦い、勝利を収める。
ダイは人間からの視線に苦悩していたが、その前に竜騎将バランが現れ、ダイが神々が作った伝説の存在「竜の騎士」であることを告げた。バランはダイに魔王軍に加わるよう促したが、ダイは拒否した。その後、ポップがメガンテを発動して犠牲になり、ダイの記憶が蘇った。レオナは世界各国の王に協力を呼びかけ、魔王軍への決戦の準備を進める。ダイ一行も新たな力を蓄え、ハドラーは妖魔司教ザボエラの手で自らを超魔生物にさせる。ダイ一行は魔王軍の本拠地である鬼岩城を破壊したが、バーンパレスには魔宮の門によって閉ざされていた。バランはダイ一行と共闘し、ハドラーと対峙するが、黒の核晶が起動してしまい、バランはダイたちを守るために全ての力を使って爆発を押さえこみながら命を落とした。
ダイ一行はバーンとの直接対決において圧倒的な力の差の前に敗北し、ハドラーがバーンに反逆し、攻撃を仕掛けた隙に脱出する。バーンはバーンパレスを飛行させ、世界各地への爆撃を実行するが、フローラの協力を得て辛うじて生き延びたダイ一行は、レオナを中心に大破邪呪文・ミナカトールを発動し、バーンパレスの動きを封じ突入する。更にハドラーと親衛騎団も乱入し、激闘を繰り広げる。ダイはハドラーを倒すが、キルバーンが仕掛けた罠がダイ達を襲う。そこに駆けつけたのは、かつてフローラから託された「カールのまもり」が身代わりとなって生還し、密かに修行を続けていたアバンであった。ダイは竜の紋章の力でバーンを追い詰め、バーンはミストバーンに預けていた肉体を使って真の姿となる。ダイは竜の魔人となり、バーンも自らの魔力の源・鬼眼を解放し、鬼眼王と化す。最後の激闘はダイの勝利で終わり、地上に帰還したダイが仲間たちに迎えられるが、キルバーンが現れ、自らの正体を明かす。キルバーンはバーンの監視と暗殺を本来の目的としていたのだった。キルバーンはダイ一行を全滅させるべく、隠していた黒の核晶を起動させて絶望する一同を嘲笑い、魔界に逃げ帰ろうとする。ダイとポップは核晶を空高く運んでいくが、土壇場でダイはポップを地上に戻し、一人爆発の閃光の中に消えた。数週間の捜索を経てもなお、ダイの姿は見つからなかったが、ロン・ベルクは持ち主の命と繋がっているダイの剣の、魔法宝玉が輝き続けていることから、ダイがどこかで生きていることを確信し、ポップはいつかダイが戻ってくる日まで、世界の平和を自分たちで守り抜くことを誓い、再びダイを探す旅に出た。
"""

In [11]:
d_index.docstore.docs['f45ce7f9-f61f-481d-988a-7db2a702842b'].text="""
孫悟空少年編では、尻尾の生えた少年・孫悟空が西の都からやってきた少女・ブルマと出会い、ドラゴンボールを7つ集めることで神龍（シェンロン）が現れることを知り、残りのドラゴンボールを探す旅に出る。その道中でウーロンやヤムチャらを巻き込んだボール探しの末、ピラフ一味にボールを奪われ神龍を呼び出されるが、ウーロンがとっさに言い放った下らない願いを叶えてもらうことで一味の野望を阻止する。悟空は武術の達人・亀仙人の下で8か月間にわたる修行を積み、その成果を確かめるために世界一の武術の達人を決める天下一武道会に出場し、準優勝を果たす。ピッコロ大魔王編では、ピラフ一味によって復活したピッコロ大魔王によって、クリリンや亀仙人など悟空の仲間たちや多くの武道家たちが殺される。悟空は仇を討つため、ヤジロベーや仙猫カリンの協力を得て命を賭して潜在する力を引き出し、ピッコロ大魔王に闘いを挑み勝利する。その後、悟空は神龍や仲間たちの復活のため天界へ向かい、ドラゴンボールの創造者である神に会う。サイヤ人編では、ピッコロ（マジュニア）との闘いから約5年後、息子の孫悟飯を儲けて平和な日々を過ごしていた悟空のもとに、実兄・ラディッツが宇宙より来襲し、自分が惑星ベジータの戦闘民族・サイヤ人であることを知らされる。
悟空は孫悟飯を助けるため、宿敵ピッコロと手を組み、自らの命と引き換えにラディッツを倒す。その後、約1年後にはさらに強力なサイヤ人たちがドラゴンボールを求めて地球に来襲することを知る。悟空は修行により増した力でナッパを一蹴し、ベジータと決闘し、仲間の協力もあり、何とか辛勝し撤退させるが、多くの仲間を失うとともに、ピッコロの戦死により彼と一心同体であった神も死亡し、地球のドラゴンボールも消滅する。悟飯たちはベジータ、フリーザ一味とのドラゴンボールをめぐる三つ巴の攻防の末、後から到着した悟空とナメック星人たちの協力を得てナメック星の神龍・ポルンガを呼び出し、ピッコロと地球のドラゴンボールを復活させる。フリーザを倒し、地球の神と殺された仲間たちを甦らせることに成功したが、その約1年後には謎の超サイヤ人によって撃退されたフリーザとその一味が再び地球を襲撃する。トランクスと名乗るその青年は、悟空は心臓病によって命を落とすこと、3年後に現れる2体の人造人間が絶望の未来をもたらすことを告げる。
セルゲームと呼ばれる武道会を開催することで、地球の命運を賭けたセルは、人造人間17号と人造人間18号を吸収して完全体となる。悟空らは、精神と時の部屋で修行を行い、セルとの闘いの中で地球を守るために命を落とす。その遺志を受け継いだ息子・悟飯が超サイヤ人2へと覚醒し、セルを撃破する。ドラゴンボールによりセルに殺された人々を蘇生させるが、悟空は自分が悪人を引き寄せているという理由で生き返りを拒否し、あの世に残ることを選ぶ。

約7年後、高校生に成長した悟飯が天下一武道会に出場することを知った悟空は、自らも出場するために占いババの力によって1日だけこの世に戻る。天下一武道会の最中、悟空たちは界王神から、恐ろしい力を持つ魔人ブウの封印が解かれようとしていることを知らされる。復活した魔人ブウにより悟飯やベジータが倒され、悟空はあの世に帰ったため、地球の命運は悟空の次男・孫悟天と少年トランクスの幼い二人に託される。魔人ブウは様々な人間との出会いからより邪悪で強力な魔人へと変貌し、悟天とトランクスが「フュージョン（融合）」して誕生した戦士・ゴテンクスや、潜在能力を解放し、パワーアップを遂げて帰ってきた悟飯らが応戦するが、苦戦を強いられる。悟空とベジータが界王神界で真の姿となった魔人ブウとの最終決戦に臨み、ドラゴンボールの協力もあり、地球・ナメック星・あの世の人々のエネルギーによって作り上げられた超特大の元気玉によって魔人ブウは完全に消滅する。

10年後、悟空は孫のパンと共に天下一武道会に久しぶりに出場し、魔人ブウの生まれ変わりである少年・ウーブと出会う。悟空はウーブと共に見果てぬ強さを追い求めて修行に旅立ち、物語は幕を閉じる。

"""

# keyword_table index

In [12]:
kt_index = GPTKeywordTableIndex([b_index, d_index], prompt_helper=prompt_helper, llm_predictor=llm_predictor)

DEBUG:root:> Adding chunk: 
十数年前、魔王ハドラー率いる魔王軍と勇者率いる勇者パーティの戦いは、勇者パーティの勝利で終わ...
DEBUG:root:> Adding chunk: ダイは魔王軍を倒すべく、ポップとゴメちゃんと共に旅立ち、ロモス王国とパプニカ王国を占拠した魔王...
DEBUG:root:> Adding chunk: バランはダイ一行と共闘し、ハドラーと対峙するが、黒の核晶が起動してしまい、バランはダイたちを守...
DEBUG:root:> Adding chunk: ダイは竜の紋章の力でバーンを追い詰め、バーンはミストバーンに預けていた肉体を使って真の姿となる...
DEBUG:root:> Adding chunk: 数週間の捜索を経てもなお、ダイの姿は見つからなかったが、ロン・ベルクは持ち主の命と繋がっている...
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["Some text is provided below. Given the text, extract up to 10 keywords from the text. Avoid stopwords.---------------------\\n\\n\\u5341\\u6570\\u5e74\\u524d\\u3001\\u9b54\\u738b\\u30cf\\u30c9\\u30e9\\u30fc\\u7387\\u3044\\u308b\\u9b54\\u738b\\u8ecd\\u3068\\u52c7\\u8005\\u7387\\u3044\\u308b\\u52c7\\u8005\\u30d1\\u30fc\\u30c6\\u30a3\\u306e\\u6226\\u3044\\u306f\\u3001\\u52c7\\u8005\\u30d1\\u30fc\\u30c6\\u30a3\\u306e\\u52dd\\u5229\\u3067\\u7d42\\u308f\\u308a\\u

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=3347 request_id=327c9657e1a1998d2da6fc3689614fa3 response_code=200
DEBUG:root:
KEYWORDS: バラン, ダイ一行, ハドラー, 黒の核晶, バーン, バーンパレス, フローラ, レオナ, 大破邪呪文, ミナカトール, キルバーン, カールのまもり, アバン
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["Some text is provided below. Given the text, extract up to 10 keywords from the text. Avoid stopwords.---------------------\\n\\u30c0\\u30a4\\u306f\\u7adc\\u306e\\u7d0b\\u7ae0\\u306e\\u529b\\u3067\\u30d0\\u30fc\\u30f3\\u3092\\u8ffd\\u3044\\u8a70\\u3081\\u3001\\u30d0\\u30fc\\u30f3\\u306f\\u30df\\u30b9\\u30c8\\u30d0\\u30fc\\u30f3\\u306b\\u9810\\u3051\\u3066\\u3044\\u305f\\u8089\\u4f53\\u3092\\u4f7f\\u3063\\u3066\\u771f\\u306e\\u59ff\\u3068\\u306a\\u308b\\u3002\\u30c0\\u30a4\\u306f

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mofuneko/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=2778 request_id=192ce1bc9bb600dfa13a72e16b711d5a response_code=200
DEBUG:root:
KEYWORDS: 孫悟空, 少年, ブルマ, ドラゴンボール, 神龍, ウーロン, ヤムチャ, 亀仙人, 修行, 天下一武道会, ピッコロ大魔王, クリリン
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["Some text is provided below. Given the text, extract up to 10 keywords from the text. Avoid stopwords.---------------------\\n\\u609f\\u7a7a\\u306f\\u4ec7\\u3092\\u8a0e\\u3064\\u305f\\u3081\\u3001\\u30e4\\u30b8\\u30ed\\u30d9\\u30fc\\u3084\\u4ed9\\u732b\\u30ab\\u30ea\\u30f3\\u306e\\u5354\\u529b\\u3092\\u5f97\\u3066\\u547d\\u3092\\u8ced\\u3057\\u3066\\u6f5c\\u5728\\u3059\\u308b\\u529b\\u3092\\u5f15\\u304d\\u51fa\\u3057\\u3001\\u30d4\\u30c3\\u30b3\\u30ed\\u5927\\u9b54\\u738b\\u306b\\u95d8\\u30

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=4228 request_id=95772666f17d414c72f4c36edea65ec0 response_code=200
DEBUG:root:
KEYWORDS: セルゲーム, 武道会, 人造人間, 悟空, 精神, 時の部屋, 修行, 地球, 悟飯, 超サイヤ人, ドラゴンボール, 悪人, 天下一武道会, 占いババ, 界王神, 魔人ブウ, 封印
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["Some text is provided below. Given the text, extract up to 10 keywords from the text. Avoid stopwords.---------------------\\n\\u5fa9\\u6d3b\\u3057\\u305f\\u9b54\\u4eba\\u30d6\\u30a6\\u306b\\u3088\\u308a\\u609f\\u98ef\\u3084\\u30d9\\u30b8\\u30fc\\u30bf\\u304c\\u5012\\u3055\\u308c\\u3001\\u609f\\u7a7a\\u306f\\u3042\\u306e\\u4e16\\u306b\\u5e30\\u3063\\u305f\\u305f\\u3081\\u3001\\u5730\\u7403\\u306e\\u547d\\u904b\\u306f\\u609f\\u7a7a\\u306e\\u6b21\\u7537\\u30fb\\u5b6b\\u6

In [13]:
kt_index.docstore.docs.keys()

dict_keys(['18482556-4c5f-4711-a484-06ad841dc8f7', 'f45ce7f9-f61f-481d-988a-7db2a702842b', '673819bb-8241-4b09-92ae-3318d71fcb01'])

In [14]:
kt_index.docstore.docs['673819bb-8241-4b09-92ae-3318d71fcb01']

KeywordTable(text=None, doc_id='673819bb-8241-4b09-92ae-3318d71fcb01', embedding=None, extra_info=None, table={'ダイ': {2099689514269512560, 5965789107349462727, 4951775241081311067, 1774055030987872190}, 'モンスター': {4951775241081311067}, 'ハドラー': {7917807643906881392, 4951775241081311067, 1774055030987872190}, 'アバン': {7917807643906881392, 4951775241081311067}, 'マホカトール': {4951775241081311067}, '平和': {2099689514269512560, 4951775241081311067}, '紋章': {4951775241081311067}, 'ポップ': {2099689514269512560, 4951775241081311067, 1774055030987872190}, '勇者': {4951775241081311067}, '力': {4951775241081311067}, 'バーンパレス': {7917807643906881392, 1774055030987872190}, '伝説': {1774055030987872190}, 'パプニカ王国': {1774055030987872190}, 'メガンテ': {1774055030987872190}, '鬼岩城': {1774055030987872190}, '竜騎将バラン': {1774055030987872190}, '魔宮': {1774055030987872190}, '神々': {1774055030987872190}, 'ゴメちゃん': {1774055030987872190}, 'ロモス王国': {1774055030987872190}, '妖魔司教ザボエラ': {1774055030987872190}, 'ミナカトール': {7917807643906881392}, 

In [15]:
from llama_index.composability import ComposableGraph
graph = ComposableGraph.build_from_index(kt_index)

In [16]:
query_configs = [
    {
        "index_struct_type": "simple_dict",
        "query_mode": "default",
        "query_kwargs": {
            "similarity_top_k": 1
        }
    },
    {
        "index_struct_type": "keyword_table",
        "query_mode": "default",
        "query_kwargs": {}
    }
]

In [17]:
graph.query("フリーザを倒したのは誰か日本語で教えて", query_configs=query_configs)
#graph.query("魔人ブウを日本語",)

INFO:root:> Starting query: フリーザを倒したのは誰か日本語で教えて
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["A question is provided below. Given the question, extract up to 10 keywords from the text. Focus on extracting the keywords that we can use to best lookup answers to the question. Avoid stopwords.\\n---------------------\\n\\u30d5\\u30ea\\u30fc\\u30b6\\u3092\\u5012\\u3057\\u305f\\u306e\\u306f\\u8ab0\\u304b\\u65e5\\u672c\\u8a9e\\u3067\\u6559\\u3048\\u3066\\n---------------------\\nProvide keywords in the following comma-separated format: \'KEYWORDS: <keywords>\'\\n"], "model": "text-davinci-003", "temperature": 0.0, "max_tokens": 1536, "top_p": 1, "frequency_penalty": 0, "presence_penalty": 0, "n": 1, "best_of": 1, "logit_bias": {}}' message='Post details'
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response'

Response(response='\n悟空がフリーザを倒したので、悟空がフリーザを倒したということになります。', source_nodes=[SourceNode(source_text='悟空は修行により増した力でナッパを一蹴し、ベジータと決闘し、仲間の協力もあり、何とか辛勝し撤退させるが、多くの仲間を失うとともに、ピッコロの戦死により彼と一心同体であった神も死亡し、地球のドラゴンボールも消滅する。悟飯たちはベジータ、フリーザ一味とのドラゴンボールをめぐる三つ巴の攻防の末、後から到着した悟空とナメック星人たちの協力を得てナメック星の神龍・ポルンガを呼び出し、ピッコロと地球のドラゴンボールを復活させる。フリーザを倒し、地球の神と殺された仲間たちを甦らせることに成功したが、その約1年後には謎の超サイヤ人によって撃退されたフリーザとその一味が再び地球を襲撃する。トランクスと名乗るその青年は、悟空は心臓病によって命を落とすこと、3年後に現れる2体の人造人間が絶望の未来をもたらすことを告げる', doc_id='f45ce7f9-f61f-481d-988a-7db2a702842b', extra_info=None, node_info={'start': 638, 'end': 991}, similarity=None), SourceNode(source_text='フリーザの持つ圧倒的な力の前にはベジータやピッコロ、悟空すら歯が立たず仲間たちが次々と命を落としていった。怒りを爆発させた悟空は伝説の戦士・超（スーパー）サイヤ人へと覚醒。フルパワーを解放したフリーザに勝利する。ポルンガによって地球に帰還した悟飯たちは復活したドラゴンボールによりサイヤ人やフリーザ一味に殺された人々を蘇生させた。一方の悟空も爆発するナメック星を辛くも脱出、ヤードラット星に漂着し一命を取り留めた。\n\n人造人間・セル編\nナメック星での闘いから約1年後、密かに生き延びていたフリーザとその一味が地球を襲撃するが、謎の超サイヤ人によって撃退される', doc_id='3c95a769-45fb-4b92-8c5c-5273bad79d1d', extra_info=None, node_info={'start': 1635, 'end': 1914}, similarity=0.

In [18]:
graph.query("ハドラーは最後どうなったのか日本語で教えて", query_configs=query_configs)

INFO:root:> Starting query: ハドラーは最後どうなったのか日本語で教えて
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["A question is provided below. Given the question, extract up to 10 keywords from the text. Focus on extracting the keywords that we can use to best lookup answers to the question. Avoid stopwords.\\n---------------------\\n\\u30cf\\u30c9\\u30e9\\u30fc\\u306f\\u6700\\u5f8c\\u3069\\u3046\\u306a\\u3063\\u305f\\u306e\\u304b\\u65e5\\u672c\\u8a9e\\u3067\\u6559\\u3048\\u3066\\n---------------------\\nProvide keywords in the following comma-separated format: \'KEYWORDS: <keywords>\'\\n"], "model": "text-davinci-003", "temperature": 0.0, "max_tokens": 1536, "top_p": 1, "frequency_penalty": 0, "presence_penalty": 0, "n": 1, "best_of": 1, "logit_bias": {}}' message='Post details'
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='Open

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=2625 request_id=a0199ed8bc4c502473ce0a3d12d56cdb response_code=200
DEBUG:root:
ハドラーは最後、ダイと共に魔王軍を撃破し、パプニカ王国を救いました。
DEBUG:root:> Initial response: 
ハドラーは最後、ダイと共に魔王軍を撃破し、パプニカ王国を救いました。
INFO:root:> [query] Total LLM token usage: 527 tokens
INFO:root:> [query] Total embedding token usage: 28 tokens
DEBUG:root:> Searching in chunk: ダイは魔王軍を倒すべく、ポップとゴメちゃんと共に旅立ち、ロモス王国とパプニカ王国を占拠した魔王...
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings
DEBUG:openai:api_version=None data='{"input": ["\\u30cf\\u30c9\\u30e9\\u30fc\\u306f\\u6700\\u5f8c\\u3069\\u3046\\u306a\\u3063\\u305f\\u306e\\u304b\\u65e5\\u672c\\u8a9e\\u3067\\u6559\\u3048\\u3066"], "encoding_format": "base64"}' message='Post details'
DEBUG:urllib3.connectionpool:https://api.op

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=2080 request_id=001f93a71c3054030e671e8681834ef6 response_code=200
DEBUG:root:

ハドラーは最後、ダイと共に魔王軍を撃破し、パプニカ王国を救い出し、そして魔王を倒したのです。
DEBUG:root:> Refined response: 

ハドラーは最後、ダイと共に魔王軍を撃破し、パプニカ王国を救い出し、そして魔王を倒したのです。
INFO:root:> [query] Total LLM token usage: 2223 tokens
INFO:root:> [query] Total embedding token usage: 84 tokens


Response(response='\n\nハドラーは最後、ダイと共に魔王軍を撃破し、パプニカ王国を救い出し、そして魔王を倒したのです。', source_nodes=[SourceNode(source_text='バランはダイ一行と共闘し、ハドラーと対峙するが、黒の核晶が起動してしまい、バランはダイたちを守るために全ての力を使って爆発を押さえこみながら命を落とした。\nダイ一行はバーンとの直接対決において圧倒的な力の差の前に敗北し、ハドラーがバーンに反逆し、攻撃を仕掛けた隙に脱出する。バーンはバーンパレスを飛行させ、世界各地への爆撃を実行するが、フローラの協力を得て辛うじて生き延びたダイ一行は、レオナを中心に大破邪呪文・ミナカトールを発動し、バーンパレスの動きを封じ突入する。更にハドラーと親衛騎団も乱入し、激闘を繰り広げる。ダイはハドラーを倒すが、キルバーンが仕掛けた罠がダイ達を襲う。そこに駆けつけたのは、かつてフローラから託された「カールのまもり」が身代わりとなって生還し、密かに修行を続けていたアバンであった', doc_id='18482556-4c5f-4711-a484-06ad841dc8f7', extra_info=None, node_info={'start': 659, 'end': 1013}, similarity=None), SourceNode(source_text='アバンへの誤解から魔王軍に加わっていたヒュンケルもまた改心し、ダイ達の仲間となる。一行は氷炎将軍フレイザードを撃破してパプニカ王国を奪還し、フレイザードに人質にされていたレオナの救出にも成功した。ハドラーはダイ達「アバンの使徒」を倒そうと躍起になるが、魔王軍内での保身を優先するがあまり、敗北を続ける。\n\nしかし、ダイは次第に自分自身の力が人間として、あまりに超絶したものであることによる他者からの視線に苦悩するようになる。そんなダイの前に魔王軍の幹部である竜騎将バランが現れ、ダイの正体が世界の秩序を乱す悪しき存在を滅ぼすために神々が作った伝説の存在「竜（ドラゴン）の騎士」であること、さらには自らの息子であることを告げた', doc_id='efec70f2-6198-44de-bafa-f4f0602f430a', extra_info=None, node_i

In [19]:
graph.query("バランは最後どうなったのか日本語で教えて", query_configs=query_configs)

INFO:root:> Starting query: バランは最後どうなったのか日本語で教えて
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["A question is provided below. Given the question, extract up to 10 keywords from the text. Focus on extracting the keywords that we can use to best lookup answers to the question. Avoid stopwords.\\n---------------------\\n\\u30d0\\u30e9\\u30f3\\u306f\\u6700\\u5f8c\\u3069\\u3046\\u306a\\u3063\\u305f\\u306e\\u304b\\u65e5\\u672c\\u8a9e\\u3067\\u6559\\u3048\\u3066\\n---------------------\\nProvide keywords in the following comma-separated format: \'KEYWORDS: <keywords>\'\\n"], "model": "text-davinci-003", "temperature": 0.0, "max_tokens": 1536, "top_p": 1, "frequency_penalty": 0, "presence_penalty": 0, "n": 1, "best_of": 1, "logit_bias": {}}' message='Post details'
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API r

Response(response='\nバランは、ポップを蘇生した後、何処かに姿を消した。', source_nodes=[SourceNode(source_text='バランはダイ一行と共闘し、ハドラーと対峙するが、黒の核晶が起動してしまい、バランはダイたちを守るために全ての力を使って爆発を押さえこみながら命を落とした。\nダイ一行はバーンとの直接対決において圧倒的な力の差の前に敗北し、ハドラーがバーンに反逆し、攻撃を仕掛けた隙に脱出する。バーンはバーンパレスを飛行させ、世界各地への爆撃を実行するが、フローラの協力を得て辛うじて生き延びたダイ一行は、レオナを中心に大破邪呪文・ミナカトールを発動し、バーンパレスの動きを封じ突入する。更にハドラーと親衛騎団も乱入し、激闘を繰り広げる。ダイはハドラーを倒すが、キルバーンが仕掛けた罠がダイ達を襲う。そこに駆けつけたのは、かつてフローラから託された「カールのまもり」が身代わりとなって生還し、密かに修行を続けていたアバンであった', doc_id='18482556-4c5f-4711-a484-06ad841dc8f7', extra_info=None, node_info={'start': 659, 'end': 1013}, similarity=None), SourceNode(source_text='人間に迫害されて家族を引き裂かれた過去を持つバランはダイに対し、魔王軍に加わり自分と共に人間を滅ぼすよう促すもダイは拒否。圧倒的なバランの力を前にしても、あくまで抵抗するダイに対しバランは息子の記憶を消去し、赤子同然となったダイを連れ去ろうとする。しかし、最後の手段としてメガンテを発動したポップの犠牲がダイの記憶を蘇らせる。親子の激闘の末にダイの奪還を断念したバランは、ポップを蘇生した後、何処かに姿を消す。\n\n大魔王との対峙、交戦\n軍団長4人を欠き、魔王軍の戦力は大きく低下したことから、レオナは世界各国の王に協力を呼びかけ、魔王軍への決戦の準備を進める', doc_id='efec70f2-6198-44de-bafa-f4f0602f430a', extra_info=None, node_info={'start': 1320, 'end': 1599}, similarity=0.83277394

In [31]:
graph.query("バランはどうして死んだのか日本語で教えて", query_configs=query_configs)

INFO:root:> Starting query: バランはどうして死んだのか日本語で教えて
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["A question is provided below. Given the question, extract up to 10 keywords from the text. Focus on extracting the keywords that we can use to best lookup answers to the question. Avoid stopwords.\\n---------------------\\n\\u30d0\\u30e9\\u30f3\\u306f\\u3069\\u3046\\u3057\\u3066\\u6b7b\\u3093\\u3060\\u306e\\u304b\\u65e5\\u672c\\u8a9e\\u3067\\u6559\\u3048\\u3066\\n---------------------\\nProvide keywords in the following comma-separated format: \'KEYWORDS: <keywords>\'\\n"], "model": "text-davinci-003", "temperature": 0.0, "max_tokens": 1536, "top_p": 1, "frequency_penalty": 0, "presence_penalty": 0, "n": 1, "best_of": 1, "logit_bias": {}}' message='Post details'
DEBUG:urllib3.connectionpool:Resetting dropped connection: api.openai.com
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "

Response(response='\nバランは、ダイを守るために自分の力を使って魔王軍との戦いに参加した結果、戦闘で死亡した。', source_nodes=[SourceNode(source_text='バランはダイ一行と共闘し、ハドラーと対峙するが、黒の核晶が起動してしまい、バランはダイたちを守るために全ての力を使って爆発を押さえこみながら命を落とした。\nダイ一行はバーンとの直接対決において圧倒的な力の差の前に敗北し、ハドラーがバーンに反逆し、攻撃を仕掛けた隙に脱出する。バーンはバーンパレスを飛行させ、世界各地への爆撃を実行するが、フローラの協力を得て辛うじて生き延びたダイ一行は、レオナを中心に大破邪呪文・ミナカトールを発動し、バーンパレスの動きを封じ突入する。更にハドラーと親衛騎団も乱入し、激闘を繰り広げる。ダイはハドラーを倒すが、キルバーンが仕掛けた罠がダイ達を襲う。そこに駆けつけたのは、かつてフローラから託された「カールのまもり」が身代わりとなって生還し、密かに修行を続けていたアバンであった', doc_id='18482556-4c5f-4711-a484-06ad841dc8f7', extra_info=None, node_info={'start': 659, 'end': 1013}, similarity=None), SourceNode(source_text='人間に迫害されて家族を引き裂かれた過去を持つバランはダイに対し、魔王軍に加わり自分と共に人間を滅ぼすよう促すもダイは拒否。圧倒的なバランの力を前にしても、あくまで抵抗するダイに対しバランは息子の記憶を消去し、赤子同然となったダイを連れ去ろうとする。しかし、最後の手段としてメガンテを発動したポップの犠牲がダイの記憶を蘇らせる。親子の激闘の末にダイの奪還を断念したバランは、ポップを蘇生した後、何処かに姿を消す。\n\n大魔王との対峙、交戦\n軍団長4人を欠き、魔王軍の戦力は大きく低下したことから、レオナは世界各国の王に協力を呼びかけ、魔王軍への決戦の準備を進める', doc_id='efec70f2-6198-44de-bafa-f4f0602f430a', extra_info=None, node_info={'start': 1320, 'end': 1599}, si

In [32]:
graph.query("キルバーンは最後どうなったのか日本語で教えて", query_configs=query_configs)

INFO:root:> Starting query: キルバーンは最後どうなったのか日本語で教えて
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["A question is provided below. Given the question, extract up to 10 keywords from the text. Focus on extracting the keywords that we can use to best lookup answers to the question. Avoid stopwords.\\n---------------------\\n\\u30ad\\u30eb\\u30d0\\u30fc\\u30f3\\u306f\\u6700\\u5f8c\\u3069\\u3046\\u306a\\u3063\\u305f\\u306e\\u304b\\u65e5\\u672c\\u8a9e\\u3067\\u6559\\u3048\\u3066\\n---------------------\\nProvide keywords in the following comma-separated format: \'KEYWORDS: <keywords>\'\\n"], "model": "text-davinci-003", "temperature": 0.0, "max_tokens": 1536, "top_p": 1, "frequency_penalty": 0, "presence_penalty": 0, "n": 1, "best_of": 1, "logit_bias": {}}' message='Post details'
DEBUG:urllib3.connectionpool:Resetting dropped connection: api.openai.com
DEBUG:urllib3.connectionpool:https://api.

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=2967 request_id=a68cae129daf3b3ec040cec2f45fff83 response_code=200
DEBUG:root:
キルバーンはアバンとマァムに倒され、核晶の爆発を止めることができなかったため、最後には滅びてしまった。
DEBUG:root:> Initial response: 
キルバーンはアバンとマァムに倒され、核晶の爆発を止めることができなかったため、最後には滅びてしまった。
INFO:root:> [query] Total LLM token usage: 496 tokens
INFO:root:> [query] Total embedding token usage: 29 tokens
DEBUG:root:> Refine context: 
キルバーンはアバンとマァムに倒され、核晶の爆発を止めることができなかったため、最後には滅びて...
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["The original question is as follows: \\u30ad\\u30eb\\u30d0\\u30fc\\u30f3\\u306f\\u6700\\u5f8c\\u3069\\u3046\\u306a\\u3063\\u305f\\u306e\\u304b\\u65e5\\u672c\\u8a9e\\u3067\\u6559\\u3048\\u3066\\nWe have provided an existing answer: \\n\\u30ad\\u30

Response(response='\n\nキルバーンはアバンとマァムに倒され、核晶の爆発を止めることができなかったため、最後に消滅してしまった。', source_nodes=[SourceNode(source_text='ダイは竜の紋章の力でバーンを追い詰め、バーンはミストバーンに預けていた肉体を使って真の姿となる。ダイは竜の魔人となり、バーンも自らの魔力の源・鬼眼を解放し、鬼眼王と化す。最後の激闘はダイの勝利で終わり、地上に帰還したダイが仲間たちに迎えられるが、キルバーンが現れ、自らの正体を明かす。キルバーンはバーンの監視と暗殺を本来の目的としていたのだった。キルバーンはダイ一行を全滅させるべく、隠していた黒の核晶を起動させて絶望する一同を嘲笑い、魔界に逃げ帰ろうとする。ダイとポップは核晶を空高く運んでいくが、土壇場でダイはポップを地上に戻し、一人爆発の閃光の中に消えた', doc_id='18482556-4c5f-4711-a484-06ad841dc8f7', extra_info=None, node_info={'start': 1014, 'end': 1294}, similarity=None), SourceNode(source_text='\n\nエピローグ\n地上に帰還したダイが仲間たちに迎えられ、皆が勝利を喜びあう中、バーンパレスの戦いでアバンに倒されたはずのキルバーンが現れ、自らの正体を明かす。キルバーンの真の主は、魔界におけるバーンのライバルで、バーンに先んじて地上征服を目論んだものの若き日のバランとの激闘の末に封印された冥竜王ヴェルザーであり、キルバーンはバーンの監視と暗殺を本来の目的としていたのだった。キルバーンはバーンをも倒したダイ一行を全滅させるべく、隠していた黒の核晶を起動させて絶望する一同を嘲笑い、魔界に逃げ帰ろうとする。アバンとマァムがキルバーンを倒すも、核晶の爆発はもはや止められない', doc_id='efec70f2-6198-44de-bafa-f4f0602f430a', extra_info=None, node_info={'start': 2757, 'end': 3043}, similarity=0.8346901379445443), SourceNode(source_text='バランは

In [32]:
graph.query("キルバーンは最後どうなったのか日本語で教えて", query_configs=query_configs)

INFO:root:> Starting query: キルバーンは最後どうなったのか日本語で教えて
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["A question is provided below. Given the question, extract up to 10 keywords from the text. Focus on extracting the keywords that we can use to best lookup answers to the question. Avoid stopwords.\\n---------------------\\n\\u30ad\\u30eb\\u30d0\\u30fc\\u30f3\\u306f\\u6700\\u5f8c\\u3069\\u3046\\u306a\\u3063\\u305f\\u306e\\u304b\\u65e5\\u672c\\u8a9e\\u3067\\u6559\\u3048\\u3066\\n---------------------\\nProvide keywords in the following comma-separated format: \'KEYWORDS: <keywords>\'\\n"], "model": "text-davinci-003", "temperature": 0.0, "max_tokens": 1536, "top_p": 1, "frequency_penalty": 0, "presence_penalty": 0, "n": 1, "best_of": 1, "logit_bias": {}}' message='Post details'
DEBUG:urllib3.connectionpool:Resetting dropped connection: api.openai.com
DEBUG:urllib3.connectionpool:https://api.

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=2967 request_id=a68cae129daf3b3ec040cec2f45fff83 response_code=200
DEBUG:root:
キルバーンはアバンとマァムに倒され、核晶の爆発を止めることができなかったため、最後には滅びてしまった。
DEBUG:root:> Initial response: 
キルバーンはアバンとマァムに倒され、核晶の爆発を止めることができなかったため、最後には滅びてしまった。
INFO:root:> [query] Total LLM token usage: 496 tokens
INFO:root:> [query] Total embedding token usage: 29 tokens
DEBUG:root:> Refine context: 
キルバーンはアバンとマァムに倒され、核晶の爆発を止めることができなかったため、最後には滅びて...
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["The original question is as follows: \\u30ad\\u30eb\\u30d0\\u30fc\\u30f3\\u306f\\u6700\\u5f8c\\u3069\\u3046\\u306a\\u3063\\u305f\\u306e\\u304b\\u65e5\\u672c\\u8a9e\\u3067\\u6559\\u3048\\u3066\\nWe have provided an existing answer: \\n\\u30ad\\u30

Response(response='\n\nキルバーンはアバンとマァムに倒され、核晶の爆発を止めることができなかったため、最後に消滅してしまった。', source_nodes=[SourceNode(source_text='ダイは竜の紋章の力でバーンを追い詰め、バーンはミストバーンに預けていた肉体を使って真の姿となる。ダイは竜の魔人となり、バーンも自らの魔力の源・鬼眼を解放し、鬼眼王と化す。最後の激闘はダイの勝利で終わり、地上に帰還したダイが仲間たちに迎えられるが、キルバーンが現れ、自らの正体を明かす。キルバーンはバーンの監視と暗殺を本来の目的としていたのだった。キルバーンはダイ一行を全滅させるべく、隠していた黒の核晶を起動させて絶望する一同を嘲笑い、魔界に逃げ帰ろうとする。ダイとポップは核晶を空高く運んでいくが、土壇場でダイはポップを地上に戻し、一人爆発の閃光の中に消えた', doc_id='18482556-4c5f-4711-a484-06ad841dc8f7', extra_info=None, node_info={'start': 1014, 'end': 1294}, similarity=None), SourceNode(source_text='\n\nエピローグ\n地上に帰還したダイが仲間たちに迎えられ、皆が勝利を喜びあう中、バーンパレスの戦いでアバンに倒されたはずのキルバーンが現れ、自らの正体を明かす。キルバーンの真の主は、魔界におけるバーンのライバルで、バーンに先んじて地上征服を目論んだものの若き日のバランとの激闘の末に封印された冥竜王ヴェルザーであり、キルバーンはバーンの監視と暗殺を本来の目的としていたのだった。キルバーンはバーンをも倒したダイ一行を全滅させるべく、隠していた黒の核晶を起動させて絶望する一同を嘲笑い、魔界に逃げ帰ろうとする。アバンとマァムがキルバーンを倒すも、核晶の爆発はもはや止められない', doc_id='efec70f2-6198-44de-bafa-f4f0602f430a', extra_info=None, node_info={'start': 2757, 'end': 3043}, similarity=0.8346901379445443), SourceNode(source_text='バランは

In [24]:
query_configs2 = [
    {
        "index_struct_type": "simple_dict",
        "query_mode": "default",
        "query_kwargs": {
            "similarity_top_k": 2
        }
    },
    {
        "index_struct_type": "keyword_table",
        "query_mode": "default",
        "query_kwargs": {}
    }
]

In [25]:
graph.query("バランは最後どうなったのか日本語で教えて", query_configs=query_configs2)

INFO:root:> Starting query: バランは最後どうなったのか日本語で教えて
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["A question is provided below. Given the question, extract up to 10 keywords from the text. Focus on extracting the keywords that we can use to best lookup answers to the question. Avoid stopwords.\\n---------------------\\n\\u30d0\\u30e9\\u30f3\\u306f\\u6700\\u5f8c\\u3069\\u3046\\u306a\\u3063\\u305f\\u306e\\u304b\\u65e5\\u672c\\u8a9e\\u3067\\u6559\\u3048\\u3066\\n---------------------\\nProvide keywords in the following comma-separated format: \'KEYWORDS: <keywords>\'\\n"], "model": "text-davinci-003", "temperature": 0.0, "max_tokens": 1536, "top_p": 1, "frequency_penalty": 0, "presence_penalty": 0, "n": 1, "best_of": 1, "logit_bias": {}}' message='Post details'
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API r

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=4939 request_id=3cc40184636aa1b65379890ab1264365 response_code=200
DEBUG:root:

バランは、ハドラーから黒の核晶を取り除こうとするも、ミストバーンによって爆発を押さえながらも命を落とした後、ダイ一行を守るために捨て駒にされたハドラーがバーンに反逆し、攻撃を仕掛けた隙に脱出した最後に姿を消した。
DEBUG:root:> Refined response: 

バランは、ハドラーから黒の核晶を取り除こうとするも、ミストバーンによって爆発を押さえながらも命を落とした後、ダイ一行を守るために捨て駒にされたハドラーがバーンに反逆し、攻撃を仕掛けた隙に脱出した最後に姿を消した。
INFO:root:> [query] Total LLM token usage: 1172 tokens
INFO:root:> [query] Total embedding token usage: 27 tokens
INFO:root:> [query] Total LLM token usage: 1297 tokens
INFO:root:> [query] Total embedding token usage: 27 tokens


Response(response='\n\nバランは、ハドラーから黒の核晶を取り除こうとするも、ミストバーンによって爆発を押さえながらも命を落とした後、ダイ一行を守るために捨て駒にされたハドラーがバーンに反逆し、攻撃を仕掛けた隙に脱出した最後に姿を消した。', source_nodes=[SourceNode(source_text='バランはダイ一行と共闘し、ハドラーと対峙するが、黒の核晶が起動してしまい、バランはダイたちを守るために全ての力を使って爆発を押さえこみながら命を落とした。\nダイ一行はバーンとの直接対決において圧倒的な力の差の前に敗北し、ハドラーがバーンに反逆し、攻撃を仕掛けた隙に脱出する。バーンはバーンパレスを飛行させ、世界各地への爆撃を実行するが、フローラの協力を得て辛うじて生き延びたダイ一行は、レオナを中心に大破邪呪文・ミナカトールを発動し、バーンパレスの動きを封じ突入する。更にハドラーと親衛騎団も乱入し、激闘を繰り広げる。ダイはハドラーを倒すが、キルバーンが仕掛けた罠がダイ達を襲う。そこに駆けつけたのは、かつてフローラから託された「カールのまもり」が身代わりとなって生還し、密かに修行を続けていたアバンであった', doc_id='18482556-4c5f-4711-a484-06ad841dc8f7', extra_info=None, node_info={'start': 659, 'end': 1013}, similarity=None), SourceNode(source_text='人間に迫害されて家族を引き裂かれた過去を持つバランはダイに対し、魔王軍に加わり自分と共に人間を滅ぼすよう促すもダイは拒否。圧倒的なバランの力を前にしても、あくまで抵抗するダイに対しバランは息子の記憶を消去し、赤子同然となったダイを連れ去ろうとする。しかし、最後の手段としてメガンテを発動したポップの犠牲がダイの記憶を蘇らせる。親子の激闘の末にダイの奪還を断念したバランは、ポップを蘇生した後、何処かに姿を消す。\n\n大魔王との対峙、交戦\n軍団長4人を欠き、魔王軍の戦力は大きく低下したことから、レオナは世界各国の王に協力を呼びかけ、魔王軍への決戦の準備を進める', doc_id='efec70f2-6198-44de-bafa-f4f0602f43

In [29]:
graph.query("バランはどうして死んだのか日本語で教えて", query_configs=query_configs2)

INFO:root:> Starting query: バランはどうして死んだのか日本語で教えて
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["A question is provided below. Given the question, extract up to 10 keywords from the text. Focus on extracting the keywords that we can use to best lookup answers to the question. Avoid stopwords.\\n---------------------\\n\\u30d0\\u30e9\\u30f3\\u306f\\u3069\\u3046\\u3057\\u3066\\u6b7b\\u3093\\u3060\\u306e\\u304b\\u65e5\\u672c\\u8a9e\\u3067\\u6559\\u3048\\u3066\\n---------------------\\nProvide keywords in the following comma-separated format: \'KEYWORDS: <keywords>\'\\n"], "model": "text-davinci-003", "temperature": 0.0, "max_tokens": 1536, "top_p": 1, "frequency_penalty": 0, "presence_penalty": 0, "n": 1, "best_of": 1, "logit_bias": {}}' message='Post details'
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API r

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=2720 request_id=cf48b97db877fb04a48068d0eafd0325 response_code=200
DEBUG:root:

バランは、ダイを守るためにハドラーから黒の核晶を引き抜こうとした結果、爆発を押さえながらも命を落として死亡した。
DEBUG:root:> Refined response: 

バランは、ダイを守るためにハドラーから黒の核晶を引き抜こうとした結果、爆発を押さえながらも命を落として死亡した。
INFO:root:> [query] Total LLM token usage: 1154 tokens
INFO:root:> [query] Total embedding token usage: 26 tokens
INFO:root:> [query] Total LLM token usage: 1277 tokens
INFO:root:> [query] Total embedding token usage: 26 tokens


Response(response='\n\nバランは、ダイを守るためにハドラーから黒の核晶を引き抜こうとした結果、爆発を押さえながらも命を落として死亡した。', source_nodes=[SourceNode(source_text='バランはダイ一行と共闘し、ハドラーと対峙するが、黒の核晶が起動してしまい、バランはダイたちを守るために全ての力を使って爆発を押さえこみながら命を落とした。\nダイ一行はバーンとの直接対決において圧倒的な力の差の前に敗北し、ハドラーがバーンに反逆し、攻撃を仕掛けた隙に脱出する。バーンはバーンパレスを飛行させ、世界各地への爆撃を実行するが、フローラの協力を得て辛うじて生き延びたダイ一行は、レオナを中心に大破邪呪文・ミナカトールを発動し、バーンパレスの動きを封じ突入する。更にハドラーと親衛騎団も乱入し、激闘を繰り広げる。ダイはハドラーを倒すが、キルバーンが仕掛けた罠がダイ達を襲う。そこに駆けつけたのは、かつてフローラから託された「カールのまもり」が身代わりとなって生還し、密かに修行を続けていたアバンであった', doc_id='18482556-4c5f-4711-a484-06ad841dc8f7', extra_info=None, node_info={'start': 659, 'end': 1013}, similarity=None), SourceNode(source_text='人間に迫害されて家族を引き裂かれた過去を持つバランはダイに対し、魔王軍に加わり自分と共に人間を滅ぼすよう促すもダイは拒否。圧倒的なバランの力を前にしても、あくまで抵抗するダイに対しバランは息子の記憶を消去し、赤子同然となったダイを連れ去ろうとする。しかし、最後の手段としてメガンテを発動したポップの犠牲がダイの記憶を蘇らせる。親子の激闘の末にダイの奪還を断念したバランは、ポップを蘇生した後、何処かに姿を消す。\n\n大魔王との対峙、交戦\n軍団長4人を欠き、魔王軍の戦力は大きく低下したことから、レオナは世界各国の王に協力を呼びかけ、魔王軍への決戦の準備を進める', doc_id='efec70f2-6198-44de-bafa-f4f0602f430a', extra_info=None, node_info={'start': 1320, 'e

In [35]:
graph.query("キルバーンの正体を日本語で教えて", query_configs=query_configs)

INFO:root:> Starting query: キルバーンの正体を日本語で教えて
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["A question is provided below. Given the question, extract up to 10 keywords from the text. Focus on extracting the keywords that we can use to best lookup answers to the question. Avoid stopwords.\\n---------------------\\n\\u30ad\\u30eb\\u30d0\\u30fc\\u30f3\\u306e\\u6b63\\u4f53\\u3092\\u65e5\\u672c\\u8a9e\\u3067\\u6559\\u3048\\u3066\\n---------------------\\nProvide keywords in the following comma-separated format: \'KEYWORDS: <keywords>\'\\n"], "model": "text-davinci-003", "temperature": 0.0, "max_tokens": 1536, "top_p": 1, "frequency_penalty": 0, "presence_penalty": 0, "n": 1, "best_of": 1, "logit_bias": {}}' message='Post details'
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=2194 request_id=0a3244f8149319bc099b7d3f820a2044 response_code=200
DEBUG:root:
キルバーンの正体は、魔界におけるバーンのライバルである冥竜王ヴェルザーです。
DEBUG:root:> Initial response: 
キルバーンの正体は、魔界におけるバーンのライバルである冥竜王ヴェルザーです。
INFO:root:> [query] Total LLM token usage: 466 tokens
INFO:root:> [query] Total embedding token usage: 22 tokens
DEBUG:root:> Refine context: 
キルバーンの正体は、魔界におけるバーンのライバルである冥竜王ヴェルザーです。...
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["The original question is as follows: \\u30ad\\u30eb\\u30d0\\u30fc\\u30f3\\u306e\\u6b63\\u4f53\\u3092\\u65e5\\u672c\\u8a9e\\u3067\\u6559\\u3048\\u3066\\nWe have provided an existing answer: \\n\\u30ad\\u30eb\\u30d0\\u30fc\\u30f3\\u306e\\u6b63\\u4f53\\u306f\\u3001\\u9b54\\u754c\\u3

Response(response='\n\nキルバーンの正体は、『ドラゴンクエスト』シリーズに登場する魔界におけるバーンのライバルである冥竜王ヴェルザーです。', source_nodes=[SourceNode(source_text='ダイは竜の紋章の力でバーンを追い詰め、バーンはミストバーンに預けていた肉体を使って真の姿となる。ダイは竜の魔人となり、バーンも自らの魔力の源・鬼眼を解放し、鬼眼王と化す。最後の激闘はダイの勝利で終わり、地上に帰還したダイが仲間たちに迎えられるが、キルバーンが現れ、自らの正体を明かす。キルバーンはバーンの監視と暗殺を本来の目的としていたのだった。キルバーンはダイ一行を全滅させるべく、隠していた黒の核晶を起動させて絶望する一同を嘲笑い、魔界に逃げ帰ろうとする。ダイとポップは核晶を空高く運んでいくが、土壇場でダイはポップを地上に戻し、一人爆発の閃光の中に消えた', doc_id='18482556-4c5f-4711-a484-06ad841dc8f7', extra_info=None, node_info={'start': 1014, 'end': 1294}, similarity=None), SourceNode(source_text='\n\nエピローグ\n地上に帰還したダイが仲間たちに迎えられ、皆が勝利を喜びあう中、バーンパレスの戦いでアバンに倒されたはずのキルバーンが現れ、自らの正体を明かす。キルバーンの真の主は、魔界におけるバーンのライバルで、バーンに先んじて地上征服を目論んだものの若き日のバランとの激闘の末に封印された冥竜王ヴェルザーであり、キルバーンはバーンの監視と暗殺を本来の目的としていたのだった。キルバーンはバーンをも倒したダイ一行を全滅させるべく、隠していた黒の核晶を起動させて絶望する一同を嘲笑い、魔界に逃げ帰ろうとする。アバンとマァムがキルバーンを倒すも、核晶の爆発はもはや止められない', doc_id='efec70f2-6198-44de-bafa-f4f0602f430a', extra_info=None, node_info={'start': 2757, 'end': 3043}, similarity=0.8433661534035644), SourceNode(source_text

In [36]:
graph.query("フリーザの最後を日本語で教えて", query_configs=query_configs)

INFO:root:> Starting query: フリーザの最後を日本語で教えて
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["A question is provided below. Given the question, extract up to 10 keywords from the text. Focus on extracting the keywords that we can use to best lookup answers to the question. Avoid stopwords.\\n---------------------\\n\\u30d5\\u30ea\\u30fc\\u30b6\\u306e\\u6700\\u5f8c\\u3092\\u65e5\\u672c\\u8a9e\\u3067\\u6559\\u3048\\u3066\\n---------------------\\nProvide keywords in the following comma-separated format: \'KEYWORDS: <keywords>\'\\n"], "model": "text-davinci-003", "temperature": 0.0, "max_tokens": 1536, "top_p": 1, "frequency_penalty": 0, "presence_penalty": 0, "n": 1, "best_of": 1, "logit_bias": {}}' message='Post details'
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/

Response(response='\nフリーザは謎の超サイヤ人によって撃退された。', source_nodes=[SourceNode(source_text='悟空は修行により増した力でナッパを一蹴し、ベジータと決闘し、仲間の協力もあり、何とか辛勝し撤退させるが、多くの仲間を失うとともに、ピッコロの戦死により彼と一心同体であった神も死亡し、地球のドラゴンボールも消滅する。悟飯たちはベジータ、フリーザ一味とのドラゴンボールをめぐる三つ巴の攻防の末、後から到着した悟空とナメック星人たちの協力を得てナメック星の神龍・ポルンガを呼び出し、ピッコロと地球のドラゴンボールを復活させる。フリーザを倒し、地球の神と殺された仲間たちを甦らせることに成功したが、その約1年後には謎の超サイヤ人によって撃退されたフリーザとその一味が再び地球を襲撃する。トランクスと名乗るその青年は、悟空は心臓病によって命を落とすこと、3年後に現れる2体の人造人間が絶望の未来をもたらすことを告げる', doc_id='f45ce7f9-f61f-481d-988a-7db2a702842b', extra_info=None, node_info={'start': 638, 'end': 991}, similarity=None), SourceNode(source_text='フリーザの持つ圧倒的な力の前にはベジータやピッコロ、悟空すら歯が立たず仲間たちが次々と命を落としていった。怒りを爆発させた悟空は伝説の戦士・超（スーパー）サイヤ人へと覚醒。フルパワーを解放したフリーザに勝利する。ポルンガによって地球に帰還した悟飯たちは復活したドラゴンボールによりサイヤ人やフリーザ一味に殺された人々を蘇生させた。一方の悟空も爆発するナメック星を辛くも脱出、ヤードラット星に漂着し一命を取り留めた。\n\n人造人間・セル編\nナメック星での闘いから約1年後、密かに生き延びていたフリーザとその一味が地球を襲撃するが、謎の超サイヤ人によって撃退される', doc_id='3c95a769-45fb-4b92-8c5c-5273bad79d1d', extra_info=None, node_info={'start': 1635, 'end': 1914}, similarity=0.83341178880019

In [37]:
res=graph.query("フリーザの最後を日本語で教えて", query_configs=query_configs2)

INFO:root:> Starting query: フリーザの最後を日本語で教えて
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["A question is provided below. Given the question, extract up to 10 keywords from the text. Focus on extracting the keywords that we can use to best lookup answers to the question. Avoid stopwords.\\n---------------------\\n\\u30d5\\u30ea\\u30fc\\u30b6\\u306e\\u6700\\u5f8c\\u3092\\u65e5\\u672c\\u8a9e\\u3067\\u6559\\u3048\\u3066\\n---------------------\\nProvide keywords in the following comma-separated format: \'KEYWORDS: <keywords>\'\\n"], "model": "text-davinci-003", "temperature": 0.0, "max_tokens": 1536, "top_p": 1, "frequency_penalty": 0, "presence_penalty": 0, "n": 1, "best_of": 1, "logit_bias": {}}' message='Post details'
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=2787 request_id=31ca30cf765754d0df082b84ebd38976 response_code=200
DEBUG:root:

フリーザは悟空、悟飯、クリリン、ブルマ、ナメック星人の協力により、謎の超サイヤ人に撃退された。
DEBUG:root:> Refined response: 

フリーザは悟空、悟飯、クリリン、ブルマ、ナメック星人の協力により、謎の超サイヤ人に撃退された。
INFO:root:> [query] Total LLM token usage: 1168 tokens
INFO:root:> [query] Total embedding token usage: 22 tokens
INFO:root:> [query] Total LLM token usage: 1290 tokens
INFO:root:> [query] Total embedding token usage: 22 tokens


Response(response='\n\nフリーザは悟空、悟飯、クリリン、ブルマ、ナメック星人の協力により、謎の超サイヤ人に撃退された。', source_nodes=[SourceNode(source_text='悟空は修行により増した力でナッパを一蹴し、ベジータと決闘し、仲間の協力もあり、何とか辛勝し撤退させるが、多くの仲間を失うとともに、ピッコロの戦死により彼と一心同体であった神も死亡し、地球のドラゴンボールも消滅する。悟飯たちはベジータ、フリーザ一味とのドラゴンボールをめぐる三つ巴の攻防の末、後から到着した悟空とナメック星人たちの協力を得てナメック星の神龍・ポルンガを呼び出し、ピッコロと地球のドラゴンボールを復活させる。フリーザを倒し、地球の神と殺された仲間たちを甦らせることに成功したが、その約1年後には謎の超サイヤ人によって撃退されたフリーザとその一味が再び地球を襲撃する。トランクスと名乗るその青年は、悟空は心臓病によって命を落とすこと、3年後に現れる2体の人造人間が絶望の未来をもたらすことを告げる', doc_id='f45ce7f9-f61f-481d-988a-7db2a702842b', extra_info=None, node_info={'start': 638, 'end': 991}, similarity=None), SourceNode(source_text='フリーザの持つ圧倒的な力の前にはベジータやピッコロ、悟空すら歯が立たず仲間たちが次々と命を落としていった。怒りを爆発させた悟空は伝説の戦士・超（スーパー）サイヤ人へと覚醒。フルパワーを解放したフリーザに勝利する。ポルンガによって地球に帰還した悟飯たちは復活したドラゴンボールによりサイヤ人やフリーザ一味に殺された人々を蘇生させた。一方の悟空も爆発するナメック星を辛くも脱出、ヤードラット星に漂着し一命を取り留めた。\n\n人造人間・セル編\nナメック星での闘いから約1年後、密かに生き延びていたフリーザとその一味が地球を襲撃するが、謎の超サイヤ人によって撃退される', doc_id='3c95a769-45fb-4b92-8c5c-5273bad79d1d', extra_info=None, node_info={'start': 1635, 'end': 1914}, 

In [38]:
res=graph.query("フリーザの最後を日本語で教えて", query_configs=query_configs2)

INFO:root:> Starting query: フリーザの最後を日本語で教えて
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["A question is provided below. Given the question, extract up to 10 keywords from the text. Focus on extracting the keywords that we can use to best lookup answers to the question. Avoid stopwords.\\n---------------------\\n\\u30d5\\u30ea\\u30fc\\u30b6\\u306e\\u6700\\u5f8c\\u3092\\u65e5\\u672c\\u8a9e\\u3067\\u6559\\u3048\\u3066\\n---------------------\\nProvide keywords in the following comma-separated format: \'KEYWORDS: <keywords>\'\\n"], "model": "text-davinci-003", "temperature": 0.0, "max_tokens": 1536, "top_p": 1, "frequency_penalty": 0, "presence_penalty": 0, "n": 1, "best_of": 1, "logit_bias": {}}' message='Post details'
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=2958 request_id=077a47202d308d0aefbbae8fa99ff731 response_code=200
DEBUG:root:

フリーザは悟空、悟飯、クリリン、ブルマ、ナメック星人の協力により、謎の超サイヤ人に撃退された。
DEBUG:root:> Refined response: 

フリーザは悟空、悟飯、クリリン、ブルマ、ナメック星人の協力により、謎の超サイヤ人に撃退された。
INFO:root:> [query] Total LLM token usage: 1168 tokens
INFO:root:> [query] Total embedding token usage: 22 tokens
INFO:root:> [query] Total LLM token usage: 1290 tokens
INFO:root:> [query] Total embedding token usage: 22 tokens


In [40]:
res.source_nodes[0]

SourceNode(source_text='悟空は修行により増した力でナッパを一蹴し、ベジータと決闘し、仲間の協力もあり、何とか辛勝し撤退させるが、多くの仲間を失うとともに、ピッコロの戦死により彼と一心同体であった神も死亡し、地球のドラゴンボールも消滅する。悟飯たちはベジータ、フリーザ一味とのドラゴンボールをめぐる三つ巴の攻防の末、後から到着した悟空とナメック星人たちの協力を得てナメック星の神龍・ポルンガを呼び出し、ピッコロと地球のドラゴンボールを復活させる。フリーザを倒し、地球の神と殺された仲間たちを甦らせることに成功したが、その約1年後には謎の超サイヤ人によって撃退されたフリーザとその一味が再び地球を襲撃する。トランクスと名乗るその青年は、悟空は心臓病によって命を落とすこと、3年後に現れる2体の人造人間が絶望の未来をもたらすことを告げる', doc_id='f45ce7f9-f61f-481d-988a-7db2a702842b', extra_info=None, node_info={'start': 638, 'end': 991}, similarity=None)

In [41]:
res.source_nodes[1]

SourceNode(source_text='フリーザの持つ圧倒的な力の前にはベジータやピッコロ、悟空すら歯が立たず仲間たちが次々と命を落としていった。怒りを爆発させた悟空は伝説の戦士・超（スーパー）サイヤ人へと覚醒。フルパワーを解放したフリーザに勝利する。ポルンガによって地球に帰還した悟飯たちは復活したドラゴンボールによりサイヤ人やフリーザ一味に殺された人々を蘇生させた。一方の悟空も爆発するナメック星を辛くも脱出、ヤードラット星に漂着し一命を取り留めた。\n\n人造人間・セル編\nナメック星での闘いから約1年後、密かに生き延びていたフリーザとその一味が地球を襲撃するが、謎の超サイヤ人によって撃退される', doc_id='3c95a769-45fb-4b92-8c5c-5273bad79d1d', extra_info=None, node_info={'start': 1635, 'end': 1914}, similarity=0.8334117888001973)

In [42]:
res.source_nodes[2]

SourceNode(source_text='仲間の協力もあり、何とか辛勝し撤退させるが、多くの仲間を失うとともに、ピッコロの戦死により彼と一心同体であった神も死亡し、地球のドラゴンボールも消滅する。\n\nフリーザ編\n地球の神と殺された仲間たちを甦らせるため、重傷で入院中の悟空に代わり、悟飯、クリリン、ブルマの3人が神とピッコロの故郷であるナメック星へ向かう。だが、そこには地球で闘ったベジータや、界王すら畏怖する宇宙の帝王・フリーザとその一味が不老不死を求めて来襲し、ナメック星人を虐殺しながらドラゴンボールを略奪していた。悟飯たちはベジータ、フリーザ一味とのドラゴンボールをめぐる三つ巴の攻防の末、後から到着した悟空とナメック星人たちの協力を得てナメック星の神龍・ポルンガを呼び出し、ピッコロと地球のドラゴンボールを復活させる。\n\n出し抜かれて願いが叶えられなかったフリーザは激怒し、一行は対決を強いられる', doc_id='3c95a769-45fb-4b92-8c5c-5273bad79d1d', extra_info=None, node_info={'start': 1252, 'end': 1634}, similarity=0.8196579974363775)